In [172]:
import os
import string
import re
from nltk.stem import *
import nltk
import itertools
import math
import operator
import sys
from statistics import mean
from tika import parser  
import fitz
import re

In [ ]:
list_text = []
for root,dir,files in os.walk('SamplePolicyDocs'):
    for file in files:
        # print(os.path.join(root,file))
        if(os.path.join(root,file).endswith('.txt')):
            s = ""
            i = 0
            text = file1.read(os.path.join(root,file))
            text = re.split(r"\n\s+",text)
            for para in text:
                para = para.split()
                # print("NEW PARA ___",para)
                for word in para:
                    # print(word+'WORD')
                    s= s+ " " + word
                    i+=1
                    if(i>=100 and s[-1]=='.'):
                        list_text.append(s)
                        s=""
                        i=0
                list_text.append(s)
                s=""
                i=0
            for paras in list_text:
                print(paras+'\n\n')




In [200]:
# doc = fitz.open('7thEditionPolicy.txt')

print(len(text))
# for i in text:
#     print("NEW PARA ----" + i)

# print(list_text[1])
        

TypeError: from_file() got an unexpected keyword argument 'encoding'

In [137]:
# import fitz
# file1 = open('7thEditionPolicy.txt',encoding='utf8')
# # doc = fitz.open('7thEditionPolicy.txt')
# list_text = []
# text = file1.read()
# text = text.split()
# s = ""
# i = 0
# # print(text)
# for word in text:
#     # print(word)
#     s= s+ " " + word
#     i+=1
#     if(i==100):
#         list_text.append(s)
#         s=""
#         i=0
# for paras in list_text:
#     print(paras+'\n\n')
# # print(list_text[1])
        


In [138]:
from nltk.corpus import stopwords
nltk.download('stopwords')

def get_stopwords():
    stop_words = [word for word in stopwords.words('english')]
    return stop_words

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ssbad\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [139]:
def lowercasify(data):
    return data.lower()


def tokenize_and_remove_punctuations(s):##
    translator = str.maketrans('','',string.punctuation+'’')
    modified_string = s.translate(translator)
    modified_string = ''.join([i for i in modified_string if not i.isdigit()])
    return nltk.word_tokenize(modified_string)

def stem_words(tokens):##
    stemmer = PorterStemmer()
    stemmed_words = [stemmer.stem(token) for token in tokens]
    return stemmed_words

def remove_stop_words(tokens):##
    stop_words = get_stopwords()
    filtered_words = [token for token in tokens if token not in stop_words and len(token) > 2]
    return filtered_words

In [140]:
def preprocess_data(list_text):
    dataDict = {}
    for i in range(len(list_text)):
        preprocessed_list=lowercasify(list_text[i])
        tokens = tokenize_and_remove_punctuations(preprocessed_list)
        filtered_tokens = remove_stop_words(tokens)
        stemmed_tokens = stem_words(filtered_tokens)
        filtered_tokens1 = remove_stop_words(stemmed_tokens)
        dataDict[i] = filtered_tokens1
        

    return dataDict

In [141]:
preprocessed_list_text = preprocess_data(list_text)
preprocessed_list_text

{0: ['\ufeffmassachusett', 'automobil', 'insur', 'polici'],
 1: ['pleas',
  'read',
  'polici',
  'part',
  'polici',
  'page',
  'mark',
  'coverag',
  'select',
  'show',
  'type',
  'amount',
  'coverag',
  'purchas',
  'read',
  'polici',
  'check',
  'coverag',
  'select',
  'page',
  'make',
  'sure',
  'show',
  'exactli',
  'intend',
  'buy',
  'question',
  'call',
  'agent',
  'compani',
  'right',
  'away',
  'polici',
  'form',
  'approv',
  'commission',
  'insur'],
 2: ['seventh', 'edit'],
 3: ['content'],
 4: ['introduct',
  'definit',
  'agreement',
  'compulsori',
  'insur',
  'part',
  'bodili',
  'injuri',
  'part',
  'person',
  'injuri',
  'protect',
  'part',
  'bodili',
  'injuri',
  'caus',
  'uninsur',
  'auto',
  'part',
  'damag',
  'someon',
  'els',
  'properti',
  'option',
  'insur',
  'part',
  'option',
  'bodili',
  'injuri',
  'part',
  'medic',
  'payment',
  'part',
  'collis',
  'part',
  'limit',
  'collis',
  'part',
  'comprehens',
  'part',
  '

In [142]:
def get_vocabulary(data):##
    tokens = []
    for token_list in data.values():
        tokens = tokens + token_list
    fdist = nltk.FreqDist(tokens)
    return list(fdist.keys())

def generate_inverted_index(data):
    all_words = get_vocabulary(data)
    index = {}
    for word in all_words:
        for doc, tokens in data.items():
            if word in tokens :
                if word in index.keys():
                    index[word].append(doc)
                else:
                    index[word] = [doc]
    return index

In [143]:
inverted_index = generate_inverted_index(preprocessed_list_text)

In [144]:
query1 = 'insurance'

In [145]:
def preprocess_query(query):
    lowercase_query=lowercasify(query)
    query_tokens = tokenize_and_remove_punctuations(lowercase_query)
    query_filtered_tokens = remove_stop_words(query_tokens)
    query_stemmed_tokens = stem_words(query_filtered_tokens)
    query_filtered_tokens1 = remove_stop_words(query_stemmed_tokens)
    return query_filtered_tokens1

In [160]:
query = "arbitration damages responsible owner investigation"
preprocessed_query = preprocess_query(query)
preprocessed_query

['arbitr', 'damag', 'respons', 'owner', 'investig']

In [147]:
def calculate_tf(tokens):
    tf_score = {}
    for token in tokens:
        tf_score[token] = tokens.count(token)
    return tf_score

In [148]:
def calculate_idf(data):
    idf_score = {}
    N = len(data)
    all_words = get_vocabulary(data)
    for word in all_words:
        word_count = 0
        for token_list in data.values():
            if word in token_list:
                word_count += 1
        idf_score[word] = math.log10(N/word_count)
    return idf_score

def calculate_tfidf(data, idf_score):
    scores = {}
    for key,value in data.items():
        scores[key] = calculate_tf(value)
    for doc,tf_scores in scores.items():
        for token, score in tf_scores.items():
            tf = score
            idf = idf_score[token]
            tf_scores[token] = tf * idf
    return scores

In [149]:
def calculate_tfidf_query(query, idf_score):
    tf_score = calculate_tf(query)
    # for key, tf_scores in scores.items():
    for token, score in tf_score.items():
        idf = 0
        tf = score
        if token in idf_score.keys():
            idf = idf_score[token]
        tf_score[token] = tf * idf
    return tf_score

In [150]:
calculate_tf(preprocessed_query)

{'automobil': 1, 'massachusett': 1, 'contract': 1, 'polici': 1}

In [151]:
def preprocess_queries(path):
    queriesDict = {}
    queries = open(path,'r').read().split('\n')
    i = 1
    for query in queries:
        tokens = tokenize_and_remove_punctuations(query)
        filtered_tokens = remove_stop_words(tokens)
        stemmed_tokens = stem_words(filtered_tokens)
        filtered_tokens1 = remove_stop_words(stemmed_tokens)
        queriesDict[i] = filtered_tokens1
        i+=1
    return queriesDict

In [169]:
query = "will you pay or reimburse if there is collision while racing or stunting?"
preprocessed_query = preprocess_query(query)
preprocessed_query

['pay', 'reimburs', 'collis', 'race', 'stunt']

In [170]:
idf_scores = calculate_idf(preprocessed_list_text)
scores = calculate_tfidf(preprocessed_list_text,idf_scores)#dict of dict storing tf-idf score of each token in each para
query_scores = calculate_tfidf_query(preprocessed_query,idf_scores)#dict 'word' : 'tf_idf'

doc_sim = {}
# query_length = 0
# for x in query_scores.values():
#     query_length+=(x*x)
# query_length

for term,term_val in query_scores.items():
    if term in inverted_index.keys():
        # print(term,term_val)
        docs = inverted_index[term]
        # print(docs)
        for doc in docs:
            doc_length = math.sqrt(sum(x ** 2 for x in scores[doc].values()))
            doc_score = scores[doc][term]#tf_idf score of term in the doc
            query_score = term_val
            query_length = 1
            query_length = math.sqrt(sum(x ** 2 for x in query_scores.values()))
            cosine_sim = (doc_score * query_score) / (doc_length * query_length)
            if doc in doc_sim.keys():
                doc_sim[doc] += cosine_sim
            else:
                doc_sim[doc] = cosine_sim
ranked = sorted(doc_sim.items(), key=operator.itemgetter(1), reverse=True)
# ranked[0]
for i in range(min(len(ranked),5)):
    print(list_text[ranked[i][0]]+"\n\n")

# print(preprocessed_query)

# scores

 Part 7. Under this Part, we will pay for any direct and accidental damage to Collision your auto caused by a collision. It does not matter who is at fault. We will also pay for collision damage to other private passenger autos while being used by you or a household member with the consent of the owner. However, we will not pay for damage to any auto which is owned or regularly used by you or a household member unless a premium for this Part is shown for that auto on the Coverage Selections Page. We will not pay if an accident occurs while an auto covered under this Part is being operated in any prearranged or organized racing, speed, stunting or demolition contest or activity or in practice or preparation for any such contest or activity.


 In that case we may be entitled to reimbursement from that person. This Part is Compulsory. You must have limits of at least $5,000. However, you may want to buy more protection. Higher limits may be purchased if agreed upon by you and by us. Howe

In [156]:
print(query_score)


0.47712125471966244


In [157]:
calculate_tfidf_query(preprocessed_query,idf_scores)


{'automobil': 1.4449365713482611,
 'massachusett': 0.784884633042612,
 'contract': 1.8129133566428555,
 'polici': 0.47712125471966244}